In [6]:
from neo4j import GraphDatabase
import pandas as pd
import ..labdataranger as ldr

db_config = ldr.query.get_db_config()

uri = f"{db_config['uri']}:{db_config['port']}"
user = db_config['username']
password = db_config['password']
database = "instruments"


def fetch_properties_by_label(label, verbose=False):
    query = f"""
    MATCH (n:{label})
    RETURN PROPERTIES(n) AS properties
    """
    if verbose:
        print(query) 
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session(database=database) as session:
        result = session.run(query)
        data = [record["properties"] for record in result]
    driver.close()
    return pd.DataFrame(data)


def collect_properties_of_all_labels(    
    meta_entities = ['User', 
                     'System', 
                     'Acquisition', 
                     'Reconstruction']):
    meta_dict = {}
    for _me in meta_entities:
        meta_dict[_me] = fetch_properties_by_label(_me)
    return meta_dict


def analyze_properties(meta_dict):
    analysis_dict = {}
    for label, df in meta_dict.items():
        prop_analysis = {}
        for column in df.columns:
            prop_analysis[column] = df[column].value_counts(dropna=False)
        analysis_dict[label] = prop_analysis
    return analysis_dict


def summarize_property_analysis(analysis_dict):
    summary_dict = {}
    for label, properties in analysis_dict.items():
        # Initialize a list to hold summary data for each property
        summary_data = []
        for prop, counts in properties.items():
            # Prepare a summary dictionary for each property
            summary = {
                'property': prop,
                'total_value_count': counts.sum(), 
                'unique_value_count': counts.nunique(dropna=False),
                'unique_values': list(counts.keys())
            }
            summary_data.append(summary)
        
        # Convert summary data into a DataFrame
        summary_dict[label] = pd.DataFrame(summary_data).sort_values(
            by='unique_value_count', 
            ascending=False).reset_index(drop=True)
    
    return summary_dict


In [7]:
meta_dict = collect_properties_of_all_labels()
meta_analysis_dict = analyze_properties(meta_dict)
summary_dict = summarize_property_analysis(meta_analysis_dict)

In [8]:
meta_dict.keys(), meta_analysis_dict.keys(), summary_dict.keys()

(dict_keys(['User', 'System', 'Acquisition', 'Reconstruction']),
 dict_keys(['User', 'System', 'Acquisition', 'Reconstruction']),
 dict_keys(['User', 'System', 'Acquisition', 'Reconstruction']))

In [9]:
summary_dict['Acquisition']

,property,total_value_count,unique_value_count,unique_values
0,numberOfFiles,10,3,"[557, 556, 558]"
1,scanningPosition,10,3,"[244.000 mm, 241.306 mm, 244.325 mm, 240.000 m..."
2,filenamePrefix,10,2,"[1L, both, Both, 2R, none, 1R, None]"
3,scanDuration,10,2,"[0h:1m:16s, 0h:1m:15s]"
4,rotationStepDeg,10,2,"[0.649, 0.648]"
5,studyDateAndTime,10,2,"[25 Sep 2020 13h:09m:23s, 18 Sep 2020 11h:02..."
6,dataDirectory,10,2,"[D:\Results\LauraM\new study\Scan2\1L, D:\Resu..."
7,filter,10,1,[Al 0.5mm]
8,randomMovement,10,1,[OFF (5)]
9,rotationSpeed,10,1,[1]


In [10]:
meta_dict['Acquisition']

,suggestedHuCalibration,imageFormat,use360Rotation,numberOfHorizontalOffsetPositions,maximumVerticalTs,imageCropOriginX,opticalAxisLine,exposureMs,filenameIndexLength,scaledImagePixelSizeUm,...,numberOfFiles,imageRotation,numberOfColumns,flatFieldCorrection,numberOfRows,cameraBinning,referenceIntensity,scanningTrajectory,depthBits,rotationDirection
0,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,558,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
1,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,557,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
2,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,557,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
3,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,557,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
4,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,556,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
5,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,556,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
6,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,558,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
7,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,557,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
8,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,557,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
9,180000,TIFF,YES,1,5.0,0,346,90,8,40.164000,...,556,0.00000,1008,ON,672,4x4,58000,ROUND,16,CC
